In [23]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("example").getOrCreate()
file_path = 'time-series-19-covid-combined.csv'
spark_df = spark.read.csv(file_path, header=True, inferSchema=True)


23/12/21 08:26:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [13]:
spark_df.show()

+----------+--------------+--------------+---------+---------+------+
|      Date|Country/Region|Province/State|Confirmed|Recovered|Deaths|
+----------+--------------+--------------+---------+---------+------+
|2020-01-22|   Afghanistan|          NULL|        0|        0|     0|
|2020-01-23|   Afghanistan|          NULL|        0|        0|     0|
|2020-01-24|   Afghanistan|          NULL|        0|        0|     0|
|2020-01-25|   Afghanistan|          NULL|        0|        0|     0|
|2020-01-26|   Afghanistan|          NULL|        0|        0|     0|
|2020-01-27|   Afghanistan|          NULL|        0|        0|     0|
|2020-01-28|   Afghanistan|          NULL|        0|        0|     0|
|2020-01-29|   Afghanistan|          NULL|        0|        0|     0|
|2020-01-30|   Afghanistan|          NULL|        0|        0|     0|
|2020-01-31|   Afghanistan|          NULL|        0|        0|     0|
|2020-02-01|   Afghanistan|          NULL|        0|        0|     0|
|2020-02-02|   Afgha

In [14]:
spark_df = spark_df.na.drop()
spark_df.show()

+----------+--------------+--------------------+---------+---------+------+
|      Date|Country/Region|      Province/State|Confirmed|Recovered|Deaths|
+----------+--------------+--------------------+---------+---------+------+
|2020-01-22|     Australia|Australian Capita...|        0|        0|     0|
|2020-01-23|     Australia|Australian Capita...|        0|        0|     0|
|2020-01-24|     Australia|Australian Capita...|        0|        0|     0|
|2020-01-25|     Australia|Australian Capita...|        0|        0|     0|
|2020-01-26|     Australia|Australian Capita...|        0|        0|     0|
|2020-01-27|     Australia|Australian Capita...|        0|        0|     0|
|2020-01-28|     Australia|Australian Capita...|        0|        0|     0|
|2020-01-29|     Australia|Australian Capita...|        0|        0|     0|
|2020-01-30|     Australia|Australian Capita...|        0|        0|     0|
|2020-01-31|     Australia|Australian Capita...|        0|        0|     0|
|2020-02-01|

In [15]:
from pyspark.sql.functions import col, to_timestamp
# Khai báo dữ liệu
data = [list(row) for row in spark_df.collect()]

# Tiền xử lý dữ liệu
def preprocess_data(data):
    # Lựa chọn cột cần giữ lại và xóa dòng có giá trị null
    filtered_data = [row[:2] + row[3:] for row in data if all(val is not None for val in row)]

    return filtered_data

# Gọi hàm tiền xử lý
preprocessed_data = preprocess_data(data)

preprocessed_data

[[datetime.date(2020, 1, 22), 'Australia', 0, 0, 0],
 [datetime.date(2020, 1, 23), 'Australia', 0, 0, 0],
 [datetime.date(2020, 1, 24), 'Australia', 0, 0, 0],
 [datetime.date(2020, 1, 25), 'Australia', 0, 0, 0],
 [datetime.date(2020, 1, 26), 'Australia', 0, 0, 0],
 [datetime.date(2020, 1, 27), 'Australia', 0, 0, 0],
 [datetime.date(2020, 1, 28), 'Australia', 0, 0, 0],
 [datetime.date(2020, 1, 29), 'Australia', 0, 0, 0],
 [datetime.date(2020, 1, 30), 'Australia', 0, 0, 0],
 [datetime.date(2020, 1, 31), 'Australia', 0, 0, 0],
 [datetime.date(2020, 2, 1), 'Australia', 0, 0, 0],
 [datetime.date(2020, 2, 2), 'Australia', 0, 0, 0],
 [datetime.date(2020, 2, 3), 'Australia', 0, 0, 0],
 [datetime.date(2020, 2, 4), 'Australia', 0, 0, 0],
 [datetime.date(2020, 2, 5), 'Australia', 0, 0, 0],
 [datetime.date(2020, 2, 6), 'Australia', 0, 0, 0],
 [datetime.date(2020, 2, 7), 'Australia', 0, 0, 0],
 [datetime.date(2020, 2, 8), 'Australia', 0, 0, 0],
 [datetime.date(2020, 2, 9), 'Australia', 0, 0, 0],
 [

In [16]:
from pyspark.sql.functions import max, min
from pyspark.sql import functions

# DataFrame cần kiểm tra
column_to_drop = "Province/State"
spark_df = spark_df.drop(column_to_drop)
df_to_check = spark_df

# List chứa tên các cột
columns = df_to_check.columns[3:]

# Tìm giá trị max và min cho từng cột
max_min_rows = []
for col in columns:
    max_value = df_to_check.select(max(col).alias("max_value")).collect()[0]["max_value"]
    min_value = df_to_check.select(min(col).alias("min_value")).collect()[0]["min_value"]

    max_row = df_to_check.filter(functions.col(col) == max_value).limit(1)
    min_row = df_to_check.filter(functions.col(col) == min_value).limit(1)

    max_min_rows.append((col, max_row, min_row))

# Hiển thị kết quả
for col, max_row, min_row in max_min_rows:
    print(f"Column: {col}")
    print("Max Value Row:")
    max_row.show(truncate=False)
    print("Min Value Row:")
    min_row.show(truncate=False)
    print("----------------------")

Column: Recovered
Max Value Row:
+----------+--------------+---------+---------+------+
|Date      |Country/Region|Confirmed|Recovered|Deaths|
+----------+--------------+---------+---------+------+
|2020-04-16|China         |67803    |64435    |3222  |
+----------+--------------+---------+---------+------+

Min Value Row:
+----------+--------------+---------+---------+------+
|Date      |Country/Region|Confirmed|Recovered|Deaths|
+----------+--------------+---------+---------+------+
|2020-01-22|Australia     |0        |0        |0     |
+----------+--------------+---------+---------+------+

----------------------
Column: Deaths
Max Value Row:
+----------+--------------+---------+---------+------+
|Date      |Country/Region|Confirmed|Recovered|Deaths|
+----------+--------------+---------+---------+------+
|2022-04-16|China         |1197078  |0        |9110  |
+----------+--------------+---------+---------+------+

Min Value Row:
+----------+--------------+---------+---------+------+
|

In [17]:
from pyspark.sql.functions import mean, median

# Tính giá trị trung bình và trung vị cho từng cột
for col in columns:
    # Giá trị trung bình
    mean_value =df_to_check.select(mean(col)).collect()[0][0]

    # Giá trị trung vị
    median_value = df_to_check.select(median(col)).collect()[0][0]

    # In kết quả
    print(f"Column: {col}")
    print(f"Mean: {mean_value}")
    print(f"Median: {median_value}")
    print("----------------------")

Column: Recovered
Mean: 1298.3093775181305
Median: 31.0
----------------------
Column: Deaths
Mean: 108.70929022293849
Median: 3.0
----------------------


In [18]:
from pyspark.sql.functions import col, count

# Tìm giá trị phổ biến và hiếm nhất cho từng cột
common_rare_values = []
for col_name in columns:
    # Tìm giá trị phổ biến (mode)
    most_common = df_to_check.groupBy(col_name).agg(count(col_name).alias("count")).orderBy(col("count").desc()).first()

    # Tìm giá trị hiếm nhất
    least_common = df_to_check.groupBy(col_name).agg(count(col_name).alias("count")).orderBy(col("count").asc()).first()

    common_rare_values.append((col_name, most_common[col_name], most_common["count"], least_common[col_name], least_common["count"]))

# Hiển thị kết quả
for col_name, common_value, common_count, rare_value, rare_count in common_rare_values:
    print(f"Column: {col_name}")
    print(f"Most Common Value: {common_value}, Count: {common_count}")
    print(f"Least Common Value: {rare_value}, Count: {rare_count}")
    print("----------------------")

Column: Recovered
Most Common Value: 0, Count: 24186
Least Common Value: 12940, Count: 1
----------------------
Column: Deaths
Most Common Value: 0, Count: 16356
Least Common Value: 496, Count: 1
----------------------


In [24]:
from pyspark.sql import SparkSession
from itertools import combinations

# Khởi tạo SparkSession
spark = SparkSession.builder.appName("CustomApriori").getOrCreate()

# Rút trích quốc gia từ dữ liệu
transactions = [row[1] for row in preprocessed_data]

# Hàm tạo các tập con
def get_subsets(transaction):
    subsets = []
    for i in range(1, len(transaction) + 1):
        subsets.extend(combinations(transaction, i))
    return subsets

# RDD của các tập con
subsets_rdd = spark.sparkContext.parallelize(transactions).flatMap(get_subsets)

# Đếm số lần xuất hiện của mỗi tập con
count_rdd = subsets_rdd.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)

# Lọc các tập con có hỗ trợ lớn hơn ngưỡng
min_support = 3
frequent_items = count_rdd.filter(lambda x: x[1] >= min_support)

# Hiển thị kết quả
print("Frequent Items:")
print(frequent_items.collect())


23/12/21 08:26:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Frequent Items:


[(('u',), 6528), (('r',), 20400), (('a',), 56304), (('A', 'u'), 6528), (('A', 's'), 6528), (('A', 'r'), 6528), (('A', 'l'), 6528), (('u', 's'), 6528), (('u', 'r'), 6528), (('u', 'a'), 13056), (('u', 'i'), 6528), (('s', 'r'), 6528), (('s', 'l'), 6528), (('t', 'i'), 17136), (('r', 'l'), 9792), (('a', 'l'), 7344), (('l', 'a'), 10608), (('i', 'a'), 34272), (('A', 'u', 's'), 6528), (('A', 'u', 'l'), 6528), (('A', 's', 'r'), 6528), (('A', 's', 'l'), 6528), (('A', 'r', 'l'), 6528), (('A', 'r', 'i'), 6528), (('A', 'a', 'l'), 6528), (('A', 'l', 'i'), 6528), (('u', 's', 't'), 6528), (('u', 's', 'r'), 6528), (('u', 's', 'i'), 6528), (('u', 't', 'l'), 6528), (('u', 'r', 'l'), 6528), (('u', 'l', 'i'), 6528), (('s', 'r', 'a'), 13056), (('s', 'r', 'i'), 6528), (('s', 'a', 'a'), 6528), (('s', 'l', 'i'), 6528), (('t', 'a', 'l'), 6528), (('t', 'a', 'i'), 6528), (('t', 'a', 'a'), 6528), (('r', 'l', 'i'), 6528), (('r', 'l', 'a'), 9792), (('a', 'l', 'i'), 6528), (('A', 'u', 's', 'r'), 6528), (('A', 'u', 't

In [19]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import DenseVector
from pyspark.sql import functions as F
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col
import numpy as np


# Tạo phiên Spark
spark = SparkSession.builder.appName("RandomForestExample").getOrCreate()

# Tạo DataFrame từ dữ liệu
schema = ["Date", "Country", "Confirmed", "Recovered", "Deaths"]
df = spark.createDataFrame(preprocessed_data, schema=schema)

# Chuyển đổi cột "Date" thành định dạng timestamp
df = df.withColumn("Date", F.col("Date").cast("timestamp"))

# Tính toán các đặc trưng
vec_assembler = VectorAssembler(inputCols=["Confirmed"], outputCol="features")
df = vec_assembler.transform(df)

# Chọn các cột quan trọng
df = df.select("features", "Confirmed")

# Phân chia dữ liệu thành tập huấn luyện và tập kiểm thử
train_data, test_data = df.randomSplit([0.8, 0.2], seed=123)

# Assuming 'Confirmed' is the label column
test_data = test_data.withColumn("label", F.col("Confirmed"))

# Định nghĩa mô hình RandomForest sử dụng PySpark DataFrame
class RandomForest:
    def __init__(self, num_trees=10, max_depth=5, subsample_size=0.8):
        self.num_trees = num_trees
        self.max_depth = max_depth
        self.subsample_size = subsample_size
        self.trees = []

    def fit(self, data):
        for _ in range(self.num_trees):
            # Lấy mẫu dữ liệu con từ tập huấn luyện
            sampled_data = data.sample(withReplacement=True, fraction=self.subsample_size, seed=np.random.randint(1, 1000))

            # Tạo DataFrame chứa cột 'tree_prediction' cho cây quyết định
            tree_predictions = sampled_data.withColumn("tree_prediction", F.rand())

            # Thêm DataFrame vào danh sách cây
            self.trees.append(tree_predictions)

    def predict(self, data):
        # Tạo DataFrame chứa cột 'tree_prediction' cho mỗi cây
        predictions = data.withColumn("tree_prediction", F.lit(0.0))

        # Cập nhật dự đoán bằng cách kết hợp dự đoán từ tất cả các cây
        for i, tree in enumerate(self.trees):
            # Đổi tên cột 'tree_prediction' thành 'prediction_i' để tránh xung đột
            tree = tree.withColumnRenamed("tree_prediction", f"prediction_{i}")
            predictions = predictions.join(tree.select("features", f"prediction_{i}"), on="features", how="left")

        # Tính tổng dự đoán từ tất cả các cây và chia tổng cho số cây để có dự đoán trung bình
        predictions = predictions.fillna(0.0)  # Điền giá trị null bằng 0
        sum_cols = sum([F.col(f"prediction_{i}") for i in range(len(self.trees))])
        predictions = predictions.withColumn("prediction", sum_cols / len(self.trees)).drop(*[f"prediction_{i}" for i in range(len(self.trees))])

        return predictions

# Tạo mô hình
random_forest_model = RandomForest(num_trees=10, max_depth=5, subsample_size=0.8)

# Huấn luyện mô hình
random_forest_model.fit(train_data)

# Dự đoán trên tập kiểm thử
predictions = random_forest_model.predict(test_data)

# Hiển thị kết quả
predictions.show()

# Cast the cluster column to DoubleType
predictions = predictions.withColumn("prediction", col("prediction").cast("double"))


# Đóng phiên Spark
spark.stop()


23/12/21 02:07:12 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+--------+---------+-----+---------------+-------------------+
|features|Confirmed|label|tree_prediction|         prediction|
+--------+---------+-----+---------------+-------------------+
|   [0.0]|        0|    0|            0.0| 0.5510490900500561|
|   [0.0]|        0|    0|            0.0|  0.542082769968515|
|   [0.0]|        0|    0|            0.0| 0.5327409693537328|
|   [0.0]|        0|    0|            0.0| 0.5265186309756457|
|   [0.0]|        0|    0|            0.0|0.49504587546702367|
|   [0.0]|        0|    0|            0.0|  0.545629168006587|
|   [0.0]|        0|    0|            0.0|  0.519126835641263|
|   [0.0]|        0|    0|            0.0|0.48007354988309536|
|   [0.0]|        0|    0|            0.0| 0.4868842337407132|
|   [0.0]|        0|    0|            0.0| 0.5568975169732463|
|   [0.0]|        0|    0|            0.0| 0.5193099669664368|
|   [0.0]|        0|    0|            0.0|0.48714444342118285|
|   [0.0]|        0|    0|            0.0|  0.539782855

In [20]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import DenseVector
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType

# Tạo phiên Spark
spark = SparkSession.builder.appName("SVMExample").getOrCreate()

# Tạo DataFrame từ dữ liệu
schema = ["Date", "Country", "Confirmed", "Recovered", "Deaths"]
df = spark.createDataFrame(preprocessed_data, schema=schema)

# Chuyển đổi cột "Date" thành định dạng timestamp
df = df.withColumn("Date", F.col("Date").cast("timestamp"))

# Tính toán các đặc trưng
vec_assembler = VectorAssembler(inputCols=["Confirmed"], outputCol="features")
df = vec_assembler.transform(df)

# Chọn các cột quan trọng
df = df.select("features", "Confirmed")

# Phân chia dữ liệu thành tập huấn luyện và tập kiểm thử
train_data, test_data = df.randomSplit([0.8, 0.2], seed=123)

# Định nghĩa mô hình SVM sử dụng PySpark DataFrame
class SVM:
    def __init__(self, lr=0.01, num_iters=100):
        self.lr = lr
        self.num_iters = num_iters
        self.weights = None

    def fit(self, data):
        # Chuyển đổi DataFrame thành RDD để sử dụng trong quá trình tính toán
        rdd_data = data.rdd.map(lambda x: (DenseVector(x[0]), x[1]))

        # Khởi tạo trọng số ngẫu nhiên
        self.weights = DenseVector([0.0] * len(rdd_data.first()[0]))

        # Gradient Descent
        for _ in range(self.num_iters):
            gradient = rdd_data.map(lambda x: ((x[0].dot(self.weights) - x[1]) * x[0], 1)).reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))
            gradient = gradient[0] / gradient[1]
            self.weights -= self.lr * gradient

    def predict(self, data):
        predict_udf = F.udf(lambda x: float(x.dot(self.weights)), returnType=FloatType())
        return data.withColumn("prediction", F.when(predict_udf(data["features"]) >= 0.5, 1.0).otherwise(0.0))

# Tạo mô hình
svm_model = SVM(lr=0.01, num_iters=100)

# Huấn luyện mô hình
svm_model.fit(train_data)

# Dự đoán trên tập kiểm thử
predictions = svm_model.predict(test_data)

# Hiển thị kết quả với cột Date
predictions.show()

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# Cast the cluster column to DoubleType
predictions = predictions.withColumn("prediction", col("prediction").cast("double"))

# Tính F1 Score
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="Confirmed", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions)
print(f"F1 Score: {f1_score}")

# Tính Accuracy
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="Confirmed", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_accuracy.evaluate(predictions)
print(f"Accuracy: {accuracy}")

# Đóng phiên Spark
spark.stop()


23/12/21 02:07:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/trung/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/ml/linalg/__init__.py:516: RuntimeWarning: overflow encountered in add
/home/trung/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/ml/linalg/__init__.py:516: RuntimeWarning: overflow encountered in add
/home/trung/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/ml/linalg/__init__.py:516: RuntimeWarning: overflow encountered in multiply
/home/trung/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/ml/linalg/__init__.py:516: RuntimeWarning: overflow encountered in add
/home/trung/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/ml/linalg/__init__.py:516: RuntimeWarning: overflow encountered in multiply
/home/trung/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/ml/linalg/__init__.py:516: RuntimeWarning: overflow encountered in multiply


+--------+---------+----------+
|features|Confirmed|prediction|
+--------+---------+----------+
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
|   [0.0]|        0|       1.0|
+--------+---------+----------+
only showing top 20 rows



F1 Score: 0.0007837874654799151


Accuracy: 0.019993222636394442


In [21]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import Row
from pyspark.ml.linalg import DenseVector
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType

# Tạo phiên Spark
spark = SparkSession.builder.appName("LinearRegressionExample").getOrCreate()

# Tạo DataFrame từ dữ liệu
schema = ["Date", "Country", "Confirmed", "Recovered", "Deaths"]
df = spark.createDataFrame(preprocessed_data, schema=schema)

# Chuyển đổi cột "Date" thành định dạng timestamp
df = df.withColumn("Date", F.col("Date").cast("timestamp"))

# Tính toán các đặc trưng
vec_assembler = VectorAssembler(inputCols=["Confirmed"], outputCol="features")
df = vec_assembler.transform(df)

# Chọn các cột quan trọng
df = df.select("features", "Confirmed")

# Phân chia dữ liệu thành tập huấn luyện và tập kiểm thử
train_data, test_data = df.randomSplit([0.8, 0.2], seed=123)

# Định nghĩa mô hình hồi quy tuyến tính sử dụng Gradient Descent
class LinearRegressionGD:
    def __init__(self, lr=0.01, num_iters=100):
        self.lr = lr
        self.num_iters = num_iters
        self.weights = None

    def fit(self, data):
        # Chuyển đổi DataFrame thành RDD để sử dụng trong quá trình tính toán
        rdd_data = data.rdd.map(lambda x: (DenseVector(x[0]), x[1]))

        # Khởi tạo trọng số ngẫu nhiên
        self.weights = DenseVector([0.0] * len(rdd_data.first()[0]))

        # Gradient Descent
        for _ in range(self.num_iters):
            gradient = rdd_data.map(lambda x: ((x[0].dot(self.weights) - x[1]) * x[0], 1)).reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))
            gradient = gradient[0] / gradient[1]
            self.weights -= self.lr * gradient

    def predict(self, data):
        predict_udf = F.udf(lambda x: float(x.dot(self.weights)), returnType=FloatType())
        return data.withColumn("prediction", predict_udf(data["features"]))

# Tạo mô hình
linear_regression_model = LinearRegressionGD(lr=0.01, num_iters=100)

# Huấn luyện mô hình
linear_regression_model.fit(train_data)

# Dự đoán trên tập kiểm thử
predictions = linear_regression_model.predict(test_data)

# Hiển thị kết quả
predictions.show()

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# Cast the cluster column to DoubleType
predictions = predictions.withColumn("prediction", col("prediction").cast("double"))

# Tính F1 Score
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="Confirmed", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions)
print(f"F1 Score: {f1_score}")

# Tính Accuracy
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="Confirmed", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_accuracy.evaluate(predictions)
print(f"Accuracy: {accuracy}")


# Đóng phiên Spark
spark.stop()



/home/trung/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/ml/linalg/__init__.py:516: RuntimeWarning: overflow encountered in add
/home/trung/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/ml/linalg/__init__.py:516: RuntimeWarning: overflow encountered in add
/home/trung/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/ml/linalg/__init__.py:516: RuntimeWarning: overflow encountered in multiply
/home/trung/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/ml/linalg/__init__.py:516: RuntimeWarning: overflow encountered in add
/home/trung/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/ml/linalg/__init__.py:516: RuntimeWarning: overflow encountered in multiply
/home/trung/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/ml/linalg/__init__.py:516: RuntimeWarning: overflow encountered in multiply


+--------+---------+----------+
|features|Confirmed|prediction|
+--------+---------+----------+
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
|   [0.0]|        0|       NaN|
+--------+---------+----------+
only showing top 20 rows



F1 Score: 0.0


Accuracy: 0.0


In [22]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Khởi tạo phiên Spark
spark = SparkSession.builder.appName("KMeansExample").getOrCreate()

# Chọn các cột quan trọng (Date và Confirmed)
selected_columns = ["Date", "Confirmed"]
df = spark_df.select(selected_columns)

# Chuyển đổi cột "Date" thành định dạng timestamp
df = df.withColumn("Date", df["Date"].cast("timestamp"))

# Tạo VectorAssembler để kết hợp các đặc trưng vào một cột vector
vector_assembler = VectorAssembler(inputCols=["Confirmed"], outputCol="features")
df = vector_assembler.transform(df)

# Chọn số cụm bạn muốn phân loại
k = 3  # Số cụm (clusters) bạn muốn tạo

# Tạo mô hình KMeans
kmeans = KMeans(featuresCol="features", predictionCol="cluster", k=k, seed=42)
model = kmeans.fit(df)

# Dự đoán cụm cho mỗi quốc gia hoặc tỉnh thành
predictions = model.transform(df)

# Hiển thị kết quả
predictions.show()

# Đánh giá chất lượng của mô hình
evaluator = ClusteringEvaluator(predictionCol="cluster", featuresCol="features")
silhouette_score = evaluator.evaluate(predictions)
print(f"Silhouette Score: {silhouette_score}")


from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# Cast the cluster column to DoubleType
predictions = predictions.withColumn("cluster", col("cluster").cast("double"))

# Tính F1 Score
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="Confirmed", predictionCol="cluster", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions)
print(f"F1 Score: {f1_score}")

# Tính Accuracy
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="Confirmed", predictionCol="cluster", metricName="accuracy")
accuracy = evaluator_accuracy.evaluate(predictions)
print(f"Accuracy: {accuracy}")


# Đóng phiên Spark
spark.stop()


23/12/21 02:15:16 ERROR Instrumentation: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.waitForCommands(Client

Py4JJavaError: An error occurred while calling o5067.fit.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.wai